In [1]:
import pandas as pd
import requests
from datetime import datetime
from dateutil import parser

from urllib import parse
from data_pull import data_pull_date
from data_pull import data_pull_all

### File Codes

In [2]:
# Energy Generation Type (All)
# "f93d1835-75bc-43e5-84ad-12472b180a98"
# https://data.nationalgrideso.com/carbon-intensity1/historic-generation-mix/r/historic_gb_generation_mix

# Energy Demand (2019)
# dd9de980-d724-415a-b344-d8ae11321432
# https://data.nationalgrideso.com/demand/historic-demand-data/r/historic_demand_data_2019

# "18c69c42-f20d-46f0-84e9-e279045befc6" (2021)


# Energy Cost (2019-2020 Financial Year)
# 6a03c415-475f-4005-bf55-b9108291c511
# https://data.nationalgrideso.com/trade-data/historic-gtma-grid-trade-master-agreement-trades-data/r/historic_gtma_trades_data_-_apr_2019_-_march_2020

# System Frequency (Dec 2019)
# f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9    
# https://data.nationalgrideso.com/system/system-frequency-data/r/december_2019_-_historic_frequency_data

# Metered Wind Power (2019 - 2020)
#c36868d9-9d43-47ce-ac3e-de69f0bb6222
#https://data.nationalgrideso.com/generation/monthly-operational-metered-wind-output/r/monthly_operational_metered_wind_output_2019-2020

### Load Generation Data

In [9]:
date_start = "2021-11-01T00:00:00.000Z"
date_end = "2022-10-31T23:59:59.999Z"
entry_limit = '100000'

#path = "C:\\Users\\kmp57\\Desktop\\Documents\\GitHub\\grid-uk-ml\\data\\"
path = "\\\\ad\\ap\\homes$\\kmp57\\My Documents\\GitHub\\grid-uk-ml\\data\\"

datacode_gen1 = "f93d1835-75bc-43e5-84ad-12472b180a98"

## Load Data
datevar = "DATETIME"
gen = data_pull_date(datacode_gen1,datevar,date_start,date_end,entry_limit)
gen[datevar] = pd.to_datetime(gen[datevar], format="%Y/%m/%d %H:%M:%S")

## Check Data Start and End times
start_time = gen.loc[0,datevar]
start_str = "Start Time: {}".format(start_time)
print(start_str)

idx = gen.index[-1]
end_time = gen.loc[idx,datevar]
end_str = "End Time: {}".format(end_time)
print(end_str)

# period = datetime.strptime("00:30:00", '%H:%M:%S').time()
# period_str = "Period: {}".format(period)
# print(period_str)     
# print(gen.size)

## Seperate Wind Data by Month and save each
T = 48 #Number of 30 min in day
idx_list = gen.index[0::T] # Find index for each day
dtm = gen.loc[idx_list,datevar] # Get each date of measurement

dtm_unique = []
for ii in dtm:
    dtm_unique.append(ii.strftime("%Y-%m"))
dtm_unique = set(dtm_unique) # Find each unique month
# print(dtm_unique)


# Assign dataframe key
dtm = gen[datevar].dt.strftime("%Y-%m") # Get all dates in dataset
gen = gen.set_index(datevar)
gen.index.names = ['Datetime']

## Save Data
for ii in dtm_unique:    
    date_match = [ii==j for j in dtm]
    gen_month =  gen.loc[date_match,:] #Extract and save all data for each unique month
    
    filename = path + "Generation\\gen_" + ii + ".csv"
    gen_month.to_csv(filename)

Start Time: 2021-11-01 00:00:00+00:00
End Time: 2022-10-31 23:30:00+00:00


In [10]:
print(gen)

                          RENEWABLE_perc NUCLEAR HYDRO_perc STORAGE  \
Datetime                                                              
2021-11-01 00:00:00+00:00           55.8  5584.0        1.9       0   
2021-11-01 00:30:00+00:00           55.7  5580.0        1.9       0   
2021-11-01 01:00:00+00:00           56.2  5583.0        1.9       0   
2021-11-01 01:30:00+00:00           56.0  5587.0        2.0       0   
2021-11-01 02:00:00+00:00           57.0  5592.0        2.0       0   
...                                  ...     ...        ...     ...   
2022-10-31 21:30:00+00:00           49.4  4624.0        2.6     296   
2022-10-31 22:00:00+00:00           51.2  4626.0        2.5     280   
2022-10-31 22:30:00+00:00           53.0  4631.0        2.1     280   
2022-10-31 23:00:00+00:00           54.7  4632.0        1.9       0   
2022-10-31 23:30:00+00:00           54.0  4633.0        1.9       0   

                          IMPORTS_perc SOLAR_perc LOW_CARBON BIOMASS_perc  \

###  Load Demand Data

In [62]:
date_start = "2021-11-01T00:00:00.000Z"
date_end = "2022-10-31T23:59:59.999Z"
entry_limit = '100000'

#datacode_dem1 = "dd9de980-d724-415a-b344-d8ae11321432"
datacode_dem =  ["18c69c42-f20d-46f0-84e9-e279045befc6","bb44a1b5-75b1-4db2-8491-257f23385006"]
datevar = 'SETTLEMENT_DATE'

## Load Data
for code in datacode_dem:
    #dem = data_pull_all(datacode2,entry_limit)
    dem = data_pull_date(code,datevar,date_start,date_end,entry_limit)
    dem[datevar] = pd.to_datetime(dem[datevar], format="%Y/%m/%d %H:%M:%S")

    ## Check Data Start and End times
    start_time = dem.loc[0,datevar]
    start_str = "Start Time: {}".format(start_time)
    print(start_str)

    idx = dem.index[-1]
    end_time = dem.loc[idx,datevar]
    end_str = "End Time: {}".format(end_time)
    print(end_str)

    # period = datetime.strptime("00:30:00", '%H:%M:%S').time()
    # period_str = "Period: {}".format(period)
    # print(period_str)   
    # print(dem.size)



    ## Seperate Demand Data by Month and save each
    T = 48 #Number of 30 min in day
    idx_list = dem.index[0::T] # Find index for each day
    dtm = dem.loc[idx_list,datevar] # Get each date of measurement

    dtm_unique = []
    for ii in dtm:
        dtm_unique.append(ii.strftime("%Y-%m"))
    dtm_unique = set(dtm_unique) # Find each unique month
    # print(dtm_unique)

    # Assign dataframe key
    dtm = dem[datevar].dt.strftime("%Y-%m") # Get all dates in dataset   
    
    timevar = "SETTLEMENT_PERIOD"
    dem[timevar] = pd.to_numeric(dem[timevar])
    dem[timevar] = dem.SETTLEMENT_PERIOD/2*60 - 30
    
    dem[datevar] = dem[datevar] + dem[timevar].astype('timedelta64[m]')
    dem = dem.drop(timevar, axis=1)
    
    dem = dem.set_index(datevar)
    dem.index.names = ['Datetime']

    ## Save Data
    for ii in dtm_unique:    
        date_match = [ii==j for j in dtm]
        dem_month =  dem.loc[date_match,:] #Extract and save all data for each unique month
        
        filename = path + "Demand\\dem_" + ii + ".csv"
        dem_month.to_csv(filename)

Start Time: 2021-11-01 00:00:00
End Time: 2021-12-31 00:00:00
Start Time: 2022-01-01 00:00:00
End Time: 2022-10-31 00:00:00


### Load Cost Data

In [5]:
## Really complicated to handle

# datacode3 = "6a03c415-475f-4005-bf55-b9108291c511"
# cst = data_pull_all(datacode3,entry_limit)

# start_time = parser.parse(cst.loc[0,"Start Time"])
# start_str = "Start Time: {}".format(start_time)
# print(start_str)

# idx = cst.index[-1]
# end_time = parser.parse(cst.loc[idx,"Start Time"])
# end_str = "End Time: {}".format(end_time)
# print(end_str) 

# cst.dtypes

### Load Frequency Data

In [6]:
# datacode4 = "f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9"
# entry_limit = '3000000'

# freq = data_pull_all(datacode4,entry_limit)
# freq['dtm'] = pd.to_datetime(freq['dtm'], format="%Y/%m/%d %H:%M:%S")

# start_time = freq.loc[0,"dtm"]
# start_str = "Start Time: {}".format(start_time)
# print(start_str)

# idx = freq.index[-1]
# end_time = freq.loc[idx,"dtm"]
# end_str = "End Time: {}".format(end_time)
# print(end_str)

# period = freq.loc[1,"dtm"] - start_time
# period_str = "Period: {}".format(period)
# print(period_str)  
# print(freq.size)
# #freq.dtypes

### Load Wind Data

In [3]:
date_start = "2021-11-01T00:00:00.000Z"
date_end = "2022-10-31T23:59:59.999Z"
entry_limit = '100000'

#path = "C:\\Users\\kmp57\\Desktop\\Documents\\GitHub\\grid-uk-ml\\data\\"
path = "\\\\ad\\ap\\homes$\\kmp57\\My Documents\\GitHub\\grid-uk-ml\\data\\"

#datacode_wind = "c36868d9-9d43-47ce-ac3e-de69f0bb6222"
datacode_wind = ["e5df03f3-a25b-4df4-a95c-8a709815924b","f732e9bb-b573-46a7-8767-3affbbb29b45"]
datevar = 'Sett_Date'

## Load Wind Data
#wind = data_pull_all(datacode_wind,entry_limit)

for code in datacode_wind:
    wind = data_pull_date(code,datevar,date_start,date_end,entry_limit)
    wind[datevar] = pd.to_datetime(wind[datevar], format="%Y/%m/%d %H:%M:%S")


    ## Check Data Start and End times
    start_time = wind.loc[0,datevar]
    start_str = "Start Time: {}".format(start_time)
    print(start_str)

    idx = wind.index[-1]
    end_time = wind.loc[idx,datevar]
    end_str = "End Time: {}".format(end_time)
    print(end_str)


    ## Seperate Wind Data by Month and save each
    T = 48 #Number of 30 min in day
    idx_list = wind.index[0::T] # Find index for each day
    dtm = wind.loc[idx_list,datevar] # Get each date of measurement

    dtm_unique = []
    for ii in dtm:
        dtm_unique.append(ii.strftime("%Y-%m"))
    dtm_unique = set(dtm_unique) # Find each unique month
    # print(dtm_unique)

    # Assign dataframe key
    dtm = wind[datevar].dt.strftime("%Y-%m") # Get all dates in wind dataset   
    
    timevar = "Sett_Period"
    wind[timevar] = pd.to_numeric(wind[timevar])
    wind[timevar] = wind[timevar]/2*60 - 30
    
    wind[datevar] = wind[datevar] + wind[timevar].astype('timedelta64[m]')
    wind = wind.drop(timevar, axis=1)
    
    wind = wind.set_index(datevar)
    wind.index.names = ['Datetime']

    
    ## Save Data
    for ii in dtm_unique:    
        date_match = [ii==j for j in dtm]
        wind_month =  wind.loc[date_match,:] #Extract and save all data for each unique month
        
        filename = path + "Wind\\wind_" + ii + ".csv"
        wind_month.to_csv(filename)

Start Time: 2021-11-01 00:00:00
End Time: 2022-03-31 00:00:00
7246
0      2021-11-01 00:00:00
1      2021-11-01 00:30:00
2      2021-11-01 01:00:00
3      2021-11-01 01:30:00
4      2021-11-01 02:00:00
               ...        
7241   2022-03-31 21:30:00
7242   2022-03-31 22:00:00
7243   2022-03-31 22:30:00
7244   2022-03-31 23:00:00
7245   2022-03-31 23:30:00
Name: Sett_Date, Length: 7246, dtype: datetime64[ns]
Start Time: 2022-04-01 00:00:00
End Time: 2022-10-31 00:00:00
10274
0       2022-04-01 00:00:00
1       2022-04-01 00:30:00
2       2022-04-01 01:00:00
3       2022-04-01 01:30:00
4       2022-04-01 02:00:00
                ...        
10269   2022-10-31 21:30:00
10270   2022-10-31 22:00:00
10271   2022-10-31 22:30:00
10272   2022-10-31 23:00:00
10273   2022-10-31 23:30:00
Name: Sett_Date, Length: 10274, dtype: datetime64[ns]


In [4]:
print(wind)

                    _count     Total England/Wales Wind Output  \
Datetime                                                         
2022-04-01 00:00:00  10274   9017.24                  7287.572   
2022-04-01 00:30:00  10274  8960.026                   7311.47   
2022-04-01 01:00:00  10274  8868.106                  7422.064   
2022-04-01 01:30:00  10274  8918.044                    7502.7   
2022-04-01 02:00:00  10274  8864.886                  7507.934   
...                    ...       ...                       ...   
2022-10-31 21:30:00  10274  9689.676                  7882.198   
2022-10-31 22:00:00  10274  9432.398                  7641.178   
2022-10-31 22:30:00  10274  9234.204                  7355.046   
2022-10-31 23:00:00  10274  9034.062                  7184.092   
2022-10-31 23:30:00  10274  8684.704                  7004.572   

                                                            _full_text    _id  \
Datetime                                                    